In [1]:
import pandas as pd
%matplotlib inline

In [2]:
path = r"C:\Users\anees\Documents\CS\MP\datasets\100k.csv"

In [3]:
df = pd.read_csv(path)

In [4]:
df.head()

,subreddit,user,comment,score
0,funny,twowordsdefault,I pretend I can manage and organize an entire ...,5737.0
1,funny,Mitoni,I finally understand why the Civ games are so ...,2100.0
2,funny,B0Boman,It appeals to the male fantasy. Also the femal...,852.0
3,funny,Scottyjscizzle,"Men only want one thing, and it's disgusting! ...",976.0
4,funny,girlsgoneoscarwilde,Science Victory or nothing. We’re either gett...,533.0


In [5]:
df.shape

(559994, 4)

In [6]:
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
import pickle

In [7]:
classifiers = []
for i in range(6):
    classifiers.append(joblib.load("lr_char_" + str(i) + ".joblib"))

In [8]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)

In [9]:
word_vectorizer = pickle.load(open("word_vectorizer.pickle",'rb'))
char_vectorizer = pickle.load(open("char_vectorizer.pickle",'rb'))

In [10]:
df = df.dropna()

In [12]:
df.comment[0]

"I pretend I can manage and organize an entire civilization when I can't manage and organize my own life a week in advance"

In [14]:
print("generating word features")
df_wordfeat = word_vectorizer.transform(df.comment)
print("creating char features")
df_charfeat = char_vectorizer.transform(df.comment)
print("done")
df_feat = hstack([df_charfeat, df_wordfeat])

generating word features


KeyboardInterrupt: 

In [13]:
df_charfeat = []
df_wordfeat = []

In [16]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [17]:
df['toxic'] = 0
df['severe_toxic'] = 0
df['obscene'] = 0
df['threat'] = 0
df['insult'] = 0
df['identity_hate'] = 0

In [18]:
i = 0
for class_name in class_names:
    df[class_name] = classifiers[i].predict_proba(df_feat)[:,1]
    i+=1

In [19]:
df_feat = 0

In [20]:
df.describe()

,score,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,559986.000000,559986.000000,559986.000000,559986.000000,559986.000000,559986.000000,559986.000000
mean,108.474133,0.130432,0.009693,0.067111,0.003554,0.053072,0.010072
std,1052.807643,0.161679,0.026782,0.136487,0.004020,0.087550,0.011501
min,-421.000000,0.002762,0.000936,0.002871,0.000937,0.002209,0.001746
25%,1.000000,0.051984,0.004887,0.023953,0.002601,0.022671,0.006815
50%,2.000000,0.078931,0.006290,0.032610,0.003057,0.031558,0.008442
75%,8.000000,0.129557,0.008222,0.047543,0.003643,0.047251,0.010561
max,75641.000000,0.999993,0.898372,0.999992,0.391820,0.999720,0.793385


In [21]:
df.to_csv("top10toxic.csv", index=False)